### **ENVIRONMENT SETUP**

In [1]:
! pip install -q emoji

     |████████████████████████████████| 240 kB 5.3 MB/s 


In [2]:
%cd /content/
! git clone https://github.com/srivarshan-s/understanding-emojis-in-tamil-emotion-detection.git
%cd understanding-emojis-in-tamil-emotion-detection/

/content
Cloning into 'understanding-emojis-in-tamil-emotion-detection'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 122 (delta 29), reused 116 (delta 28), pack-reused 0
Receiving objects: 100% (122/122), 1.76 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/understanding-emojis-in-tamil-emotion-detection


### **IMPORT LIBRARIES**

In [3]:
import numpy as np
import pandas as pd
import emoji
import re

import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report

### **STEMMER**

In [4]:
! rm input.txt
! rm output.txt
%cd snowball-with-tamil/
! make
%cd ..

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory
/content/understanding-emojis-in-tamil-emotion-detection/snowball-with-tamil
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/space.o compiler/space.c
cc -Iinclude -O2 -W -Wall -Wmissing-prototypes -Wmissing-declarations  -c -o compiler/tokeniser.o compiler/tokeniser.c
In file included from compiler/tokeniser.c:6:0:
compiler/tokeniser.c: In function ‘read_token’:
compiler/header.h:8:21: warning: this statement may fall through [-Wimplicit-fallthrough=]
 #define unless(C) if(!(C))
                     ^
compiler/tokeniser.c:390:16: note: in expansion of macro ‘unless’
                unless (t->next == 0) {
                ^~~~~~
compiler/tokeniser.c:401:13: note: here
             default:
             ^~~~~~~
compiler/tokeniser.c: In function ‘white_space’:
compiler/tokeniser.c:106:34: warning: this statement may fall through [-Wimplic

### **IMPORT DATASET**

In [5]:
header_names = ["emotion", "text"]
df_train = pd.read_csv("data/ta-emotion10-train.csv", sep="\t", names=header_names)
df_dev = pd.read_csv("data/ta-emotion10-dev.csv", sep="\t", names=header_names)
df_test = pd.read_csv("data/task_a_test.csv", sep="\t", names=header_names)

In [6]:
df = pd.concat([df_train, df_dev, df_test])

In [7]:
df.head()

,emotion,text
0,Neutral,நாளைக்கு அரிசிக்கு இந்த நிலமை வந்தா 🙂
1,Anger,மானம் கேட்ட அன்புமணி
2,Neutral,தவறு இஸ்ரேல் இருக்காது இதை நான் கூறவில்லை ஹமாஸ...
3,Joy,கொங்கு நாட்டு சிங்கம் உன்மையும் நேர்மையும் உலை...
4,Neutral,இவர் யார்? ஒவ்வொரு வார்த்தையும் முன்னுக்கு பின...


In [8]:
df.describe()

,emotion,text
count,22200,22200
unique,11,22200
top,Neutral,நாளைக்கு அரிசிக்கு இந்த நிலமை வந்தா 🙂
freq,7601,1


In [9]:
df.emotion.unique()

array(['Neutral', 'Anger', 'Joy', 'Disguist', 'Trust', 'Anticipation',
       'Ambiguous', 'Love', 'Surprise', 'Sadness', 'Fear'], dtype=object)

In [10]:
pd.value_counts(df.emotion)

Neutral         7601
Joy             3394
Ambiguous       2626
Trust           1903
Disguist        1397
Anticipation    1312
Anger           1262
Sadness         1127
Love            1060
Surprise         362
Fear             156
Name: emotion, dtype: int64

### **DATA CLEANING**

In [11]:
# Remove rows without emojis

drop_idx = []

for text, idx in zip(df.text, df.index):
    if len(emoji.distinct_emoji_list(text)) == 0:
        drop_idx.append(idx)

df.drop(df.index[drop_idx], inplace=True)

df.reset_index(inplace=True, drop=True)

In [12]:
df.describe()

,emotion,text
count,1818,1818
unique,11,1818
top,Joy,அண்ணே இங்கேயும் வந்துட்டீங்களா🤣🤣🤣😂😂
freq,585,1


In [13]:
pd.value_counts(df.emotion)

Joy             585
Neutral         401
Trust           183
Love            143
Ambiguous       139
Sadness         120
Anticipation     73
Disguist         69
Anger            59
Surprise         34
Fear             12
Name: emotion, dtype: int64

In [14]:
df.head()

,emotion,text
0,Surprise,அண்ணே இங்கேயும் வந்துட்டீங்களா🤣🤣🤣😂😂
1,Ambiguous,யாருக்கு தெரியும் பொண்ணு பார்க்க கூட குடும்பத்...
2,Ambiguous,அது என்ன 🔥பனியிடை நீக்கம் பனி நீக்கம் தான் செறி 👍
3,Neutral,தி மு க விற்க்கு எனது 7 கோடி நன்றிகள்... அந்த ...
4,Love,கணவன் அமைவதெல்லாம் இறைவன் கொடுத்த வரம் ❤️


In [15]:
# Replace emoji with emoji name

def replace_emoji_with_name(text):
    return emoji.demojize(text, delimiters=("[", "]"))

df["text"] = df["text"].apply(replace_emoji_with_name)

In [16]:
df.head()

,emotion,text
0,Surprise,அண்ணே இங்கேயும் வந்துட்டீங்களா[rolling_on_the_...
1,Ambiguous,யாருக்கு தெரியும் பொண்ணு பார்க்க கூட குடும்பத்...
2,Ambiguous,அது என்ன [fire]பனியிடை நீக்கம் பனி நீக்கம் தான...
3,Neutral,தி மு க விற்க்கு எனது 7 கோடி நன்றிகள்... அந்த ...
4,Love,கணவன் அமைவதெல்லாம் இறைவன் கொடுத்த வரம் [red_he...


### **DATA PREPROCESSING**

In [17]:
text = df['text']
label = df['emotion']

In [18]:
le = LabelEncoder()
label = le.fit_transform(label)

In [19]:
text = text.str.replace(r"[+/#@&*$%:]", '', regex=True)
text = text.to_numpy()

In [20]:
text = text.tolist()

! rm input.txt
! rm output.txt

for i in text:
    text_file = open("input.txt", "a")
    text_file.write(i + '\n')
    text_file.close()

rm: cannot remove 'input.txt': No such file or directory
rm: cannot remove 'output.txt': No such file or directory


In [21]:
! chmod +x snowball-with-tamil/stemwords
! ./snowball-with-tamil/stemwords -l ta -i input.txt -o output.txt

In [22]:
text_file = open("output.txt", "r")

text = []

for line in text_file:
    text.append(line.strip())

In [23]:
with open('stopwords/tamil_stopwords.txt', encoding = 'utf-8') as f:
    tamil_stopwords = f.readlines()
    for i in range(len(tamil_stopwords)):
        tamil_stopwords[i] = re.sub('\n','',tamil_stopwords[i])
stopwords = tamil_stopwords

In [24]:
# Function for removing stop words
def stopwords_remove(text):
    # Appending words which are not stop words
    text = text.split(" ")
    removed = [s for s in text if s not in stopwords]  
    return removed

text = [stopwords_remove(s) for s in text]
for i in range(len(text)):
  text[i] = (" ".join(text[i]))

text = np.array(text)

### **FEATURE EXTRACTION**

In [25]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(text)
X = X.toarray()
print(X.shape)

y = label
print(y.shape)

(1818, 410)
(1818,)


### **LOGISTIC REGRESSION**

In [26]:
# # Gridsearch

# parameters = {
#     "penalty": ["l1", "l2", "elasticnet", "none"],
#     "dual": [True, False],
#     "C": [1, 0.1, 0.01],
#     "fit_intercept": [True, False],
#     "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
# }

# model = LogisticRegression()

# grid_search = GridSearchCV(model, parameters, n_jobs=-1)
# grid_search.fit(X, y)

# grid_search.best_params_

In [27]:
# {'C': 1,
#  'dual': False,
#  'fit_intercept': False,
#  'penalty': 'l2',
#  'solver': 'newton-cg'}

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # model = LogisticRegression()
    model = LogisticRegression(C=1, dual=False, fit_intercept=False, 
                               penalty="l2", solver="newton-cg")
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.23      0.14      0.18       139
           1       0.42      0.19      0.26        59
           2       0.20      0.03      0.05        73
           3       0.08      0.01      0.02        69
           4       0.00      0.00      0.00        12
           5       0.42      0.76      0.54       585
           6       0.17      0.05      0.08       143
           7       0.25      0.23      0.24       401
           8       0.53      0.50      0.51       120
           9       0.00      0.00      0.00        34
          10       0.21      0.12      0.15       183

    accuracy                           0.36      1818
   macro avg       0.23      0.18      0.19      1818
weighted avg       0.30      0.36      0.31      1818



### **SUPPORT VECTOR MACHINE**

In [28]:
# {'C': 1,
#  'break_ties': True,
#  'decision_function_shape': 'ovr',
#  'degree': 2,
#  'gamma': 'scale',
#  'kernel': 'linear',
#  'probability': True,
#  'shrinking': True}

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # model = SVC()
    model = SVC(C=1, break_ties=True, decision_function_shape="ovr", degree=2, 
                gamma="scale", kernel="linear", probability=True, 
                shrinking=True)
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.30      0.14      0.19       139
           1       0.42      0.17      0.24        59
           2       0.00      0.00      0.00        73
           3       0.00      0.00      0.00        69
           4       0.00      0.00      0.00        12
           5       0.42      0.79      0.55       585
           6       0.00      0.00      0.00       143
           7       0.28      0.31      0.29       401
           8       0.54      0.46      0.50       120
           9       0.00      0.00      0.00        34
          10       0.30      0.11      0.16       183

    accuracy                           0.38      1818
   macro avg       0.20      0.18      0.17      1818
weighted avg       0.30      0.38      0.31      1818



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **NAIVE BAYES**

In [29]:
kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = GaussianNB()
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.09      0.09      0.09       139
           1       0.09      0.17      0.12        59
           2       0.06      0.21      0.10        73
           3       0.09      0.14      0.11        69
           4       0.00      0.00      0.00        12
           5       0.33      0.12      0.18       585
           6       0.14      0.32      0.20       143
           7       0.18      0.08      0.11       401
           8       0.20      0.23      0.21       120
           9       0.09      0.29      0.14        34
          10       0.15      0.15      0.15       183

    accuracy                           0.15      1818
   macro avg       0.13      0.16      0.13      1818
weighted avg       0.20      0.15      0.15      1818



### **STOCHASTIC GRADIENT DESCENT**

In [30]:
# {'alpha': 0.0001,
#  'early_stopping': False,
#  'fit_intercept': True,
#  'l1_ratio': 0.15,
#  'loss': 'perceptron',
#  'n_jobs': -1,
#  'penalty': 'l2',
#  'shuffle': True,
#  'warm_start': False}

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # model = SGDClassifier(alpha=0.0001, early_stopping=False, fit_intercept=True, l1_ratio=0.15, loss="perceptron",
    #                       n_jobs=-1, penalty="l2", shuffle=True, warm_start=False)
    model = SGDClassifier()
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.13      0.14      0.14       139
           1       0.24      0.20      0.22        59
           2       0.06      0.05      0.06        73
           3       0.12      0.09      0.10        69
           4       0.00      0.00      0.00        12
           5       0.43      0.58      0.49       585
           6       0.14      0.07      0.09       143
           7       0.24      0.19      0.22       401
           8       0.41      0.51      0.45       120
           9       0.05      0.03      0.04        34
          10       0.20      0.17      0.19       183

    accuracy                           0.31      1818
   macro avg       0.19      0.19      0.18      1818
weighted avg       0.28      0.31      0.29      1818



### **K NEAREST NEIGHBOURS**

In [31]:
# {'algorithm': 'ball_tree',
#  'leaf_size': 25,
#  'n_neighbors': 5,
#  'p': 2,
#  'weights': 'distance'}

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = KNeighborsClassifier(algorithm="ball_tree", leaf_size=25, n_neighbors=5, p=2, weights="distance")
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.15      0.32      0.20       139
           1       0.12      0.15      0.14        59
           2       0.15      0.03      0.05        73
           3       0.08      0.03      0.04        69
           4       0.00      0.00      0.00        12
           5       0.47      0.50      0.49       585
           6       0.17      0.17      0.17       143
           7       0.26      0.26      0.26       401
           8       0.49      0.47      0.48       120
           9       0.00      0.00      0.00        34
          10       0.22      0.12      0.15       183

    accuracy                           0.31      1818
   macro avg       0.19      0.19      0.18      1818
weighted avg       0.30      0.31      0.30      1818



### **DECISION TREE**

In [32]:
# {'criterion': 'gini', 'max_features': None, 'splitter': 'best'}

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = DecisionTreeClassifier(criterion="gini", max_features=None, splitter="best")
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.16      0.19      0.17       139
           1       0.19      0.22      0.20        59
           2       0.04      0.03      0.03        73
           3       0.02      0.01      0.02        69
           4       0.00      0.00      0.00        12
           5       0.45      0.51      0.48       585
           6       0.15      0.14      0.14       143
           7       0.23      0.23      0.23       401
           8       0.44      0.40      0.42       120
           9       0.04      0.03      0.03        34
          10       0.18      0.15      0.16       183

    accuracy                           0.29      1818
   macro avg       0.17      0.17      0.17      1818
weighted avg       0.27      0.29      0.28      1818



### **RANDOM FOREST**

In [33]:
# {'bootstrap': True,
#  'class_weight': None,
#  'criterion': 'entropy',
#  'max_features': 'log2',
#  'n_estimators': 100,
#  'oob_score': False,
#  'warm_start': False}

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion="entropy", max_features="log2",
    #                                n_estimators=100, oob_score=False, warm_start=False)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.20      0.13      0.16       139
           1       0.40      0.17      0.24        59
           2       0.20      0.03      0.05        73
           3       0.14      0.01      0.03        69
           4       0.00      0.00      0.00        12
           5       0.44      0.76      0.55       585
           6       0.15      0.04      0.07       143
           7       0.26      0.29      0.28       401
           8       0.50      0.45      0.47       120
           9       0.00      0.00      0.00        34
          10       0.29      0.10      0.15       183

    accuracy                           0.37      1818
   macro avg       0.23      0.18      0.18      1818
weighted avg       0.31      0.37      0.31      1818



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### **XGBOOST**

In [34]:
# {'booster': 'gbtree',
#  'grow_policy': 'depthwise',
#  'learning_rate': 0.1,
#  'max_depth': 6,
#  'sampling_method': 'uniform',
#  'tree_method': 'hist'}

kf = KFold(n_splits=5)

pred = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = xgb.XGBClassifier(booster="gbtree", grow_policy="depthwise", learning_rate=0.1, max_depth=6,
                              sampling_method=6, tree_method="hist")
    model.fit(X_train, y_train)

    pred_test = model.predict(X_test).tolist()
    pred += pred_test

print(classification_report(y, pred))

              precision    recall  f1-score   support

           0       0.28      0.16      0.20       139
           1       0.31      0.14      0.19        59
           2       0.06      0.01      0.02        73
           3       0.09      0.03      0.04        69
           4       0.00      0.00      0.00        12
           5       0.43      0.73      0.54       585
           6       0.18      0.07      0.10       143
           7       0.28      0.29      0.28       401
           8       0.49      0.40      0.44       120
           9       0.00      0.00      0.00        34
          10       0.28      0.15      0.19       183

    accuracy                           0.36      1818
   macro avg       0.22      0.18      0.18      1818
weighted avg       0.31      0.36      0.32      1818



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
